In [1]:
import numpy as np
from scipy.optimize import brentq
import pandas as pd
# use extended precision (longdouble) for discount factor calculations
np.set_printoptions(precision=18)

def market_discount_factors(spot_rates):
    # spot_rates: array-like of length n with decimals (e.g. 0.03)
    # use the notebook's default numpy precision (float64)
    spot = np.asarray(spot_rates, dtype=np.float64)
    n = len(spot)
    discounts = np.empty(n, dtype=np.float64)
    for k in range(n):
        discounts[k] = 1.0 / np.power(1.0 + spot[k], k + 1)
    return discounts

def price_zcb_on_tree(a_list, b, t):
    # a_list: list/array of a_1..a_t (length >= t)
    # b: scalar
    # t: integer maturity (1..len(a_list))
    # returns model P(0,t) for unit face
    # Node indexing: time k has k+1 nodes j=0..k
    # Terminal prices at time t:
    P_next = np.ones(t+1)  # payoff = 1 at maturity
    for k in range(t, 0, -1):
        a_k = a_list[k-1]
        # short rates r(k,j) for j=0..k
        r_k = a_k * np.exp(b * np.arange(0, k+1))
        P_prev = np.empty(k)
        # backward one step
        for j in range(k):
            P_prev[j] = 0.5 * (P_next[j] / (1.0 + r_k[j])) + 0.5 * (P_next[j+1] / (1.0 + r_k[j+1]))
        P_next = P_prev
    return P_next[0]

def calibrate_bdt_a(spot_rates, b=0.05, tol=1e-8):
    # spot_rates: list of length n (decimals)
    n = len(spot_rates)
    P_market = market_discount_factors(spot_rates)  # length n
    a_cal = np.zeros(n)
    for k in range(1, n+1):
        # objective: f(a) = model_price(0,k) - P_market[k-1]
        def f(a_val):
            a_try = a_cal.copy()
            a_try[k-1] = a_val
            return price_zcb_on_tree(a_try, b, k) - P_market[k-1]

        # bracket search
        low, high = 1e-12, 0.5
        f_low, f_high = f(low), f(high)
        # expand high until sign change or cap
        ntries = 0
        while f_low * f_high > 0 and ntries < 80:
            high *= 2.0
            f_high = f(high)
            ntries += 1
        if f_low * f_high > 0:
            raise RuntimeError(f"Could not bracket root for a_{k}: f(low)={f_low}, f(high)={f_high}")

        a_root = brentq(f, low, high, xtol=1e-12, rtol=1e-12, maxiter=200)
        # ensure residual small enough
        if abs(f(a_root)) > tol:
            raise RuntimeError(f"Calibration failed at k={k}, residual {abs(f(a_root))} > {tol}")
        a_cal[k-1] = a_root
    return a_cal

In [2]:
def build_bdt_short_rate_lattice(a_params, b):
    """
    Build a right-aligned BDT short-rate lattice as a pandas DataFrame.
    a_params: 1D array-like of a_1..a_n (length n)
    b: scalar
    returns: n x n DataFrame with NaN in unused entries
    """
    n = len(a_params)
    rates = np.full((n, n), np.nan, dtype=float)
    for col in range(n):
        for j in range(col + 1):
            row = n - 1 - j
            rates[row, col] = a_params[col] * np.exp(b * j)
    return pd.DataFrame(rates, index=range(n), columns=range(n))

In [3]:
def compute_elementary_prices(lattice, q):
    rows, cols, size  = lattice.shape[0]+1, lattice.shape[1]+1, lattice.shape[0]+1
    A = pd.DataFrame(np.nan, index=range(rows), columns=range(cols), dtype=float)
    A.iat[rows-1, 0] = 1.0  # time 0 state
    A
    for i in range(1, size): #columns
        for k in range(size-1,size-i-2,-1): #rows
            if k == size-1:
                A.iat[k,i] = (1-q)*A.iat[k,i-1]/(1 + lattice.iat[size-2,i-1])
            elif (k != size-1 and k+i == size-1):
                #A.iat[k,i] = k + i
                A.iat[k,i] = (q)*A.iat[k+1,i-1]/(1 + lattice.iat[size-2,i-1])  
            elif (k != size-1 and k+i != size-1):
                A.iat[k,i] = (q)*A.iat[k+1,i-1]/(1 + lattice.iat[size-2,i-1]) + (1-q)*A.iat[k,i-1]/(1 + lattice.iat[size-2,i-1])
    # append a row named "Model Prices" containing column-wise sums (NaNs ignored)
    col_sums = A.sum(axis=0)
    A.loc['Model Prices'] = col_sums

    # append a row named "Market Spot Rates" with calculated values
    #market_spot_rates = 100 * ((1 / A.loc['Model Prices'][1:]) ** (1 / A.index[1:]) - 1)
    #A.loc['Market Spot Rates', 1:] = market_spot_rates
    return A

In [8]:
spot_rates = [0.073, 0.0762,0.081,0.0845, 0.092,0.0964,0.1012,0.1045,0.1075, 0.1122,0.1155,0.1192,0.122,0.1232]
short_rate_lattice = build_bdt_short_rate_lattice(spot_rates, b=0.005)
A = compute_elementary_prices(short_rate_lattice, q=0.5)
A
#model_prices = market_discount_factors(spot_rates)
#model_prices


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000016
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000036,0.000224
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000081,0.000467,0.001454
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000180,0.000966,0.002799,0.005815
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000402,0.001983,0.005315,0.010264,0.015991
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000895,0.004022,0.009914,0.017717,0.025659,0.031983
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001981,0.008051,0.018097,0.029743,0.039863,0.046187,0.047974
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004377,0.015852,0.032205,0.048260,0.059486,0.063781,0.061583,0.054828
8,NaN,NaN,NaN,NaN,NaN,NaN,0.009640,0.030640,0.055481,0.075144,0.084454,0.083281,0.074411,0.061583,0.047974
9,NaN,NaN,NaN,NaN,NaN,0.021139,0.057840,0.091919,0.110963,0.112716,0.101345,0.083281,0.063781,0.046187,0.031983


In [105]:
spot_rates_test = [0.03,0.031,0.032,0.033,0.034,0.035,0.0355,0.036,0.0365,0.037]
market_discount_factors(spot_rates_test)
calibrate_bdt_a(spot_rates_test, b=0.05, tol=1e-12)
#build_bdt_short_rate_lattice(a_params, b=0.05)


array([0.029250688474689375, 0.03042298053509584 , 0.03152016040733118 ,
       0.03254540565579816 , 0.033501789470989855, 0.03439228296950313 ,
       0.03227295945705709 , 0.032291027502625524, 0.03228946458436631 ,
       0.032269310685631575])

In [9]:
short_rate_lattice

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.131474
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.129544,0.130818
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.125940,0.128898,0.130166
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.121422,0.125312,0.128255,0.129517
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.117364,0.120816,0.124687,0.127615,0.128871
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.111887,0.116779,0.120214,0.124065,0.126979,0.128228
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.108222,0.111329,0.116197,0.119614,0.123446,0.126346,0.127588
7,NaN,NaN,NaN,NaN,NaN,NaN,0.104282,0.107682,0.110774,0.115617,0.119017,0.122830,0.125715,0.126952
8,NaN,NaN,NaN,NaN,NaN,0.098840,0.103762,0.107145,0.110221,0.115040,0.118424,0.122218,0.125088,0.126319
9,NaN,NaN,NaN,NaN,0.093859,0.098347,0.103244,0.106611,0.109672,0.114467,0.117833,0.121608,0.124465,0.125689


In [ ]:
swaption_fixed_rate = 0.1165
option_expiration = 2
swap_maturity = 10
option_strike = 0
principal = 1000000
q = 0.5

swap_lattice = pd.DataFrame(np.nan, index=range(swap_maturity), columns=range(swap_maturity), dtype=float)
swap_lattice.iloc[:, -1] = ((short_rate_lattice[swap_maturity-1].dropna() - swaption_fixed_rate) / (1 + short_rate_lattice[swap_maturity-1].dropna())).reset_index(drop=True)
swap_lattice

short_rate_lattice_trim = short_rate_lattice.iloc[-swap_maturity:, :swap_maturity].reset_index(drop=True)
short_rate_lattice_trim
for i in range(swap_maturity - 2,-1,-1): #cols
    for k in range(swap_maturity-1,swap_maturity -2 - i,-1): #rows
        swap_lattice.iat[k,i] = (short_rate_lattice_trim.iat[k,i] - swaption_fixed_rate)/(1+short_rate_lattice_trim.iat[k,i]) + (q*swap_lattice.iat[k-1,i+1] + (1-q)*swap_lattice.iat[k,i+1])/(1+short_rate_lattice_trim.iat[k,i]) 
        #print(k,i)  
swap_lattice

swap_lattice_trim = swap_lattice.iloc[-(option_expiration+1):, :(option_expiration+1)].reset_index(drop=True)
swap_lattice_trim



,0,1,2
0,NaN,NaN,-0.107393
1,NaN,-0.138120,-0.110166
2,-0.170651,-0.141098,-0.112935


,0,1,2,3,4,5,6,7,8,9
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.117364
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.111887,0.116779
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.108222,0.111329,0.116197
3,NaN,NaN,NaN,NaN,NaN,NaN,0.104282,0.107682,0.110774,0.115617
4,NaN,NaN,NaN,NaN,NaN,0.098840,0.103762,0.107145,0.110221,0.115040
5,NaN,NaN,NaN,NaN,0.093859,0.098347,0.103244,0.106611,0.109672,0.114467
6,NaN,NaN,NaN,0.085777,0.093390,0.097857,0.102729,0.106079,0.109125,0.113896
7,NaN,NaN,0.081814,0.085349,0.092925,0.097369,0.102217,0.105550,0.108580,0.113328
8,NaN,0.076582,0.081406,0.084924,0.092461,0.096883,0.101707,0.105024,0.108039,0.112762
9,0.073,0.076200,0.081000,0.084500,0.092000,0.096400,0.101200,0.104500,0.107500,0.112200


,0,1,2,3,4,5,6,7,8,9
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000774
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.003688,0.000250
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.011237,-0.004663,-0.000272
3,NaN,NaN,NaN,NaN,NaN,NaN,-0.021861,-0.012608,-0.005634,-0.000791
4,NaN,NaN,NaN,NaN,NaN,-0.036749,-0.023583,-0.013975,-0.006601,-0.001309
5,NaN,NaN,NaN,NaN,-0.055223,-0.038780,-0.025300,-0.015337,-0.007565,-0.001825
6,NaN,NaN,NaN,-0.080218,-0.057530,-0.040806,-0.027012,-0.016695,-0.008526,-0.002338
7,NaN,NaN,-0.107393,-0.082767,-0.059832,-0.042827,-0.028720,-0.018050,-0.009484,-0.002849
8,NaN,-0.138120,-0.110166,-0.085312,-0.062130,-0.044844,-0.030423,-0.019399,-0.010438,-0.003359
9,-0.170651,-0.141098,-0.112935,-0.087853,-0.064423,-0.046855,-0.032122,-0.020745,-0.011388,-0.003866


InvalidIndexError: (9, 8)

In [24]:
(short_rate_lattice[swap_maturity-1].dropna()-swaption_fixed_rate)/(1+short_rate_lattice[swap_maturity-1].dropna())

4     0.000774
5     0.000250
6    -0.000272
7    -0.000791
8    -0.001309
9    -0.001825
10   -0.002338
11   -0.002849
12   -0.003359
13   -0.003866
Name: 9, dtype: float64